<a href="https://colab.research.google.com/github/Gary-Schulke/Project3Startup/blob/master/SQL_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [0]:
connection = psycopg2.connect(
    host = 'myprojectdb.cqbbih9hnlbh.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'root',
    password = 'finalproject',
    database='root'
    )
cursor=connection.cursor()

In [0]:
sql = """
SELECT a.state, a.state_case, a.veh_crash, a.pedestrians, a.motorists_involved, a.rural_urban, a.latitude, a.longitude, \
a.fatalities, a.drunk_drivers, a.city, a.day, a.month, a.year, a.hour, a.minute, a.light_conditions, d.distract_num, i.imparement_num, \
p.person_num, p.age, p.sex, p.injury_severity, p.alc_involved, p.alc_test_result, p.drugs, p.drug_num, v.violation_num
FROM accident a 
JOIN distract d ON a.state_case = d.state_case AND a.year = d.year
JOIN drimpair i ON a.state_case = i.state_case AND a.year = i.year
JOIN person p ON a.state_case = p.state_case AND a.year = p.year
JOIN violation v ON a.state_case = v.state_case AND a.year = v.year
WHERE a.year = 2012
"""
accident = pd.read_sql(sql, con=connection)

In [0]:
target = accident["fatalities"]
target_names = [0,1,2,3,4,5]

In [17]:
accidentdf = accident.drop("fatalities", axis=1)
accidentdf = accidentdf.drop("state", axis=1)
accidentdf = accidentdf.drop("state_case", axis=1)
feature_names = accidentdf.columns
accidentdf

,veh_crash,pedestrians,motorists_involved,rural_urban,latitude,longitude,drunk_drivers,city,day,month,year,hour,minute,light_conditions,distract_num,imparement_num,person_num,age,sex,injury_severity,alc_involved,alc_test_result,drugs,drug_num,violation_num
0,1,0,1,1,36.895261,-120.654,1,0,11,6,2012,99,99,2,0,9,1,61,1,4,1,18,0,1,0
1,1,0,1,1,36.895261,-120.654,1,0,11,6,2012,99,99,2,0,9,1,61,1,4,1,18,0,1,0
2,1,0,1,1,34.208839,-117.457,1,0,2,7,2012,18,42,1,0,99,1,44,1,4,9,35,9,600,0
3,1,0,1,1,34.208839,-117.457,1,0,2,7,2012,18,42,1,0,99,1,44,1,4,9,35,9,600,0
4,1,0,1,1,34.492969,-118.179,0,0,3,7,2012,15,45,1,0,0,1,63,1,4,0,96,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4826943,3,0,6,2,34.047100,-118.447,0,1980,31,1,2012,20,25,3,1,95,1,31,1,0,0,0,0,0,0
4826944,1,0,1,1,40.855519,-120.760,0,0,20,5,2012,13,48,1,96,99,1,47,1,4,9,0,9,1,0
4826945,1,0,1,1,40.855519,-120.760,0,0,20,5,2012,13,48,1,96,99,1,47,1,4,9,0,9,1,0
4826946,1,0,1,2,32.692739,-117.015,0,0,16,6,2012,12,5,1,0,99,1,39,1,4,9,0,9,1,0


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(accident, target, random_state=42)

In [19]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [20]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

1.0

In [21]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.5101234170254437, 'day'),
 (0.08267206579939615, 'motorists_involved'),
 (0.06516967748146517, 'hour'),
 (0.06456521471371317, 'latitude'),
 (0.04272200325216919, 'city'),
 (0.039253579435785405, 'year'),
 (0.036642141464486626, 'distract_num'),
 (0.03031235947828242, 'month'),
 (0.027841842440875033, 'imparement_num'),
 (0.02729542242109454, 'drunk_drivers'),
 (0.019867826824237145, 'pedestrians'),
 (0.01626404609399064, 'minute'),
 (0.009612653437739939, 'veh_crash'),
 (0.007074660739885581, 'age'),
 (0.005568646137791076, 'longitude'),
 (0.004087208103756844, 'person_num'),
 (0.001590765148392812, 'drug_num'),
 (0.0015182441391688225, 'violation_num'),
 (0.0014137736520191484, 'alc_involved'),
 (0.0013149356939923173, 'drugs'),
 (0.0008961684543246416, 'injury_severity'),
 (0.0005477100383820817, 'alc_test_result'),
 (0.00044998751235591155, 'rural_urban'),
 (0.0004428504274034359, 'sex'),
 (0.0, 'light_conditions')]